# Pre-processing Notebook 1 for clients data
<br>
<em> Modified GettingStarted notebook from the master branch to censor/redact clients on the basis of their first sleep date and subsequently filtering the clients first n days of records(n = 30 , 60, 90, 120, 150, 90 days) . The first sleep date is the first Stay of a client in the database. </em> 

### Using Data Analytics Utility Functions of GettingStarted
- utility code found in `util/`.

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
"""Standard routines for pre-processing and analyzing client data from the Calgary
Drop-In Centre.
"""

import os
import pandas as pd
import numpy as np


def RemoveByStartDate(tbl,winStartDate,winEndDate,dateSelect = pd.Series(dtype='object')):
    """Remove all records for subjects in tbl first appearing in the data between
    winStartDate and winEndDate (determined using tbl.Date values)."""

    if dateSelect.empty:
        tblFlt = tbl
    else:
        tblFlt = tbl.loc[dateSelect]
        
    startDates = tblFlt.groupby('ClientId').apply(lambda x: min(x.Date))
    notCensored = ~ ((startDates >= winStartDate) & (startDates <= winEndDate ))
    
    return tbl.loc[tbl.ClientId.isin(startDates[notCensored].index)]    

def ShelterGroupDemographics(tbl):
    """Summarizes the demographics of a group of shelter clients.
    - Fields:
     > TotalStays: Total number of shelter stays.
     > Tenure: Number of days between first and last appearance in dataset.
     > UsagePct: Percentage of days during tenure spent in shelter.
     > AvgGapLen: Average length of gaps between shelter stays (days).
                  NaN for clients with a single stay.
     > TotalEpisodes: Total number of episodes of shelter access.
     """
    
    dates = tbl.Date.drop_duplicates().sort_values() 
    tl = pd.DataFrame({
        'Date': dates,                
        'Ind': range(1,len(dates)+1)  
        })
    
    tenure = (tl.Date.max() - tl.Date.min()).days + 1
    gapVals = tl.Date.diff().astype('timedelta64[D]')
    nStays = tl.Ind.max()
    
    return pd.Series({
        'Tenure': tenure,  # Total span of days a client interacts with shelter.
        'UsagePct': 100.0*nStays/tenure,  # Percentage of days during tenure client stayed in shelter.
        'AvgGapLen': gapVals.mean(),  # Average length of gaps in shelter stays.
        'TotalStays': nStays,  # Total number of shelter stays.
        'TotalEpisodes': sum(gapVals >= episodeGap)+1  # Total number of episodes.
    })

def CalculateStaySequence(tbl):
    """Determines a stay timeline for a subject.
    - Each event in the timeline is represented by an index and a timestamp.
    - A stay is defined as accessing one or more services (typically sleep services) 
      in a 24 hour period.
    - Timestamps generated using tbl.Date values."""
    
    dates = tbl.Date.drop_duplicates().sort_values() # Drop duplicates since stay is one or more sleep.
    return pd.DataFrame({
        'Date': dates,                 # Date of each stay.
        'Ind': range(1,len(dates)+1)   # Index of each stay.
    })


episodeGap = 30  # The max gap in stays before a new episode is created.

def CalculateEpisodeSequence(tbl):    
    """Determines an episode timeline for a subject.  
    - Each event in the timeline is represented by an index and a timestamp.
    - An episode is a series of shelter stays separated by gaps of less than 
      di_data.episodeGap days.
    - A stay is defined as accessing one or more services (typically sleep services) 
      in a 24 hour period.
    - Timestamps generated using tbl.Date values."""
    
    stayDates = tbl.Date.drop_duplicates().sort_values() # Drop duplicates since stay is one or more sleep.
    gapVals = stayDates.diff().astype('timedelta64[D]')
    gapInd = (gapVals >= episodeGap).astype('int').cumsum().drop_duplicates(keep='first')
    
    return pd.DataFrame({
        'Date': tbl.loc[gapInd.index].Date, # Date of first day of each episode.
        'Ind': range(1,len(gapInd)+1)       # Episode index.
    })


def TimeWinThresholdTest(tbl,posFlag,negFlag,thresh,winSzDays):
    """Analyze a subject timeline and determine if the number of events
    exceed thresh in a time window of winSzDays.
    - idDate is the date the threshold test is satisfied.
    - reqTime is the number of days it took to satisfy the threshold test.
    - If the test is satisfied, return a series with posFlag, idDate and reqTime.
    - If the test is not satisfied, return a series with negFlag and nan values
      for idDate and reqTime."""
    
    win = tbl.rolling('{:d}d'.format(winSzDays),on='Date').count().Ind
    
    registrationDate = tbl.Date.min()
    idDate = tbl[win >= thresh].Date.min()  # Will be equal to NaN if the threshold isn't met.
    reqTime = (idDate - registrationDate).days
    
    if idDate == idDate:   # Satisfied if idDate is not NaN.
        return pd.Series({
            'Flag': posFlag,  
            'Date': idDate,  # Date subject was identified.
            'Time': reqTime  # Number of days it took to identify subject.
        })
    else:
        return pd.Series({   # Returned if the test is not satisfied.
            'Flag': negFlag,
            'Date': pd.NaT,
            'Time': np.nan
        })

    
def ChooseEarliestTest(test1,test2):
    """Merges two test tables.  If each test is positive for a subject, the test that
    occurs earliest in a subject's timeline is chosen.  
    
    If you have more than two test tables, you can call this routine several times.  For 
    example, tables A, B and C can be merged by:
      mrg = ChooseEarliestTest(A,B)
      mrg = ChooseEarliestTest(mrg,C)
      
    Assumptions:
    - Both test tables contain the identical list of subjects.
    - Both tests use the same flag for a negative result.
    """
    nRec = len(test1.index)
    
    tbl = pd.DataFrame({ 'Flag': ['']*nRec, 'Date': [pd.NaT]*nRec, 'Time': [np.nan]*nRec },index=test1.index)

    bothNeg = (test1.Time != test1.Time) & (test2.Time != test2.Time)
    tbl[bothNeg] = test1[bothNeg]
    
    isOne = (test1.Time == test1.Time) & (test2.Time == test2.Time) & (test1.Time < test2.Time)
    isOne = isOne | ( (test1.Time == test1.Time) & (test2.Time != test2.Time) )
    tbl[isOne] = test1[isOne]
    
    isTwo = (test1.Time == test1.Time) & (test2.Time == test2.Time) & (test1.Time >= test2.Time)
    isTwo = isTwo | ( (test2.Time == test2.Time) & (test1.Time != test1.Time) )
    tbl[isTwo] = test2[isTwo]
    
    return tbl

In [ ]:
import numpy as np
import pandas as pd
import datetime, copy, imp
import time

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

## Coding Style
- In general, we will follow the [CamelCase](https://en.wikipedia.org/wiki/Camel_case) naming convention.
- Functions, classes, data types and pandas column names will follow upper camel case or Pascal case where the first letter is capitalized (ie. PrintOutput(), ReturnResults()).
- Variable names will follow lower camel case or Dromedary case where the first letter is not capitalized (ie. myVariable, functionReturnValue).
- Notebook names should be descriptive and can use typical sentence capitalization and spacing.
- Module names should be all lower case with underscores optionally used for word separation. 

## Censoring
When analyzing event timelines, our data will always have a finite window with a start and stop time. *Censoring* refers to when a value occurs that is outside the range of a measuring instrument.  For a study with a finite time window:
- *Left Censoring*: Occurs when a subject's activity starts before the study window.  
- *Right Censoring*: Occurs when a subject's activity continues after the end of the study window.

If mitigating censoring is important, a buffer period can be defined at the start and end of the study window.  Any subjects with time records in the start buffer are removed (addresses left censoring) and any subjects with time records in the end buffer are removed.

**IMPORTANT:** All data records for the subjects that appear in the start/end buffers are removed, not just the records that occur in the buffer.  Censored subjects must be removed entirely from the dataset.

In [ ]:
help(RemoveByStartDate)

## Pre-Processing
- In general, you will likely need to develop a pre-processing routine that is specific to your research.  It will take in the master dataset and perform a series of modifications to the data that all of your routines will require.  In this case, decorating your pre-processing function with the local caching routine CacheResult will prevent the pre-processing routine from having to run each time you execute your notebook.

The following pre-processing example is for a study that utilizes machine learning to predict chronic homelessness based on client age and number of sleep stays in shelter.

In [ ]:
def PreProcessSleepEntries():
    
    tbl = pd.read_hdf('UniversityExportAnonymized.hd5')
    tbl.drop_duplicates(subset=['ClientId', 'EntryType', 'Date'], keep='first', inplace=True)
    tbl = tbl[tbl.EntryType == 'Sleep'][['ClientId','Date','EntryType']]
    
    leftStart = tbl.Date.min()
    leftEnd = pd.to_datetime('2009-07-01')
    rightStart = pd.to_datetime('2017-07-31')
    rightEnd = tbl.Date.max()
    
    nClientsAll = len(tbl.ClientId.unique())
    
    tbl = RemoveByStartDate(tbl,leftStart,leftEnd)
    nLeftRemoved = nClientsAll - len(tbl.ClientId.unique())

    tbl = RemoveByStartDate(tbl,rightStart,rightEnd)
    nRightRemoved = nClientsAll - nLeftRemoved - len(tbl.ClientId.unique())

    nClients = len(tbl.ClientId.unique())
 
    print('Total Valid Clients: {:d}/{:d} ({:d} removed left, {:d} removed right)'
          .format(nClients,nClientsAll,nLeftRemoved,nRightRemoved))

    return tbl

In [ ]:
housing = PreProcessSleepEntries()

In [ ]:
# Dataframe housing only contains the dates of sleep entries of the valid clients

In [ ]:
dfShelter = pd.read_hdf('UniversityExportAnonymized.hd5')    
totalClients = len(dfShelter.ClientId.unique())
print('Total Clients:',totalClients)

### First Sleep Date
 
- It is seprate from Registration Date. It is the first time a client stays at the DI-shelter and accesses Day/Night sleep facilities (or both)
- Registration date is defined seprately in this notebook

In [ ]:
def FindFirstSleepDate(tbl):
    return min(tbl.Date)
sleepDates = housing.groupby('ClientId').progress_apply(FindFirstSleepDate)

In [ ]:
sleepDates

### Registration Date 
- Registration date is the First time a particular client accesses the DI-shelter for any service (sleep,counselor,laundry etc)
- Registration date is the first date a client shows up in the dataset.

In [ ]:
def FindRegistrationDate(tbl):
    return min(tbl.Date)
registrationDate = dfShelter.groupby('ClientId').progress_apply(FindRegistrationDate)

In [ ]:
sleepDates = pd.DataFrame(sleepDates)
registrationDate = pd.DataFrame(registrationDate)
sleepDates = sleepDates.rename(columns={0:'FirstSleepDate'})
registrationDate = registrationDate.rename(columns={0:'RegistrationDate'})

In [ ]:
mergeDates = registrationDate.join(sleepDates, how ='left',on='ClientId')
mergeDates

### Valid Clients
- These are the valid client ID's Registration Date and First Sleep Date
- Redaction done on the basis of first Sleep date in PreProcessSleepEntries

In [ ]:
mergeDates[mergeDates.FirstSleepDate.notnull()]

### Filtering out Clients who never Stay at DI shelter
- The following code filters out invalid client's shelter data who dont have even a single sleep entry on the basis of first Sleep date
- By Replacing the NaN values of FirstSleepDate with an arbitrary date before the left-cutoff so that the clients who never access the sleep facilities of the DI Shelter are also redacted using the left-cutoff

In [ ]:
arbitaryDate = pd.to_datetime('2001-01-01')
mergeDates = mergeDates.fillna(arbitaryDate)

In [ ]:
leftCutoff = pd.to_datetime('2009-07-01')
rightCutoff = pd.to_datetime('2017-07-31')
validClientInd = (mergeDates.FirstSleepDate >= leftCutoff) & (mergeDates.FirstSleepDate <= rightCutoff)
nClientsAll = 34577
nClients = len(sleepDates)
print('Original Clients: %d'% (totalClients))
print('Total clients who access Sleep facilities: %d \nValid Clients: %d \nValid Clients per original clients: (%.1f%%) \nValid Clients per total clients acessing sleep facilities: (%.1f%%)' % (nClientsAll,nClients,100*nClients/totalClients,100*nClients/nClientsAll))

### Redaction (Censoring) of the clients records on the basis of their first sleep date
- Include only valid clients that meet the left/right censoring cutoff criteria and access DI: all entry types including sleep, counsellor, logs etc. 

In [ ]:
validClientsDf = dfShelter.loc[list(validClientInd[dfShelter.ClientId])]

In [ ]:
len(validClientsDf.ClientId.unique()) # To verify check if this number matches the number of Valid Clients

In [ ]:
validClientsDf.drop_duplicates(subset=['ClientId', 'EntryType', 'Date'], keep='first', inplace=True)

In [ ]:
df_sorted = validClientsDf.sort_values(['ClientId', 'Date'])

<h3>Saving the records of the censored clients to the disk</h3>

In [ ]:
validClientsDf.loc[:,'Location'] = validClientsDf['Location'].astype(str)
validClientsDf.loc[:,'EntryType'] = validClientsDf['EntryType'].astype(str)
validClientsDf.loc[:,'ClientState'] = validClientsDf['ClientState'].astype(str)

In [ ]:
df = validClientsDf.sort_values(['ClientId', 'Date'])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

start_date = pd.to_datetime('2009-07-01')
end_date = pd.to_datetime('2017-07-31')
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
filtered_df = df.loc[mask]

In [ ]:
filtered_df.to_hdf('validClientsDf.h5', key='df', mode='w')